# `rorcid`

`rorcid` is a package developed by [Scott Chamberlain](https://scottchamberlain.info/), co-founder of [rOpenSci](https://ropensci.org/), to serve as an interface to the ORCID API. 

You can find more information about the API [on the ORCID site](http://members.orcid.org/api/about-public-api).

Credit to Paul Oldham at https://www.pauloldham.net/introduction-to-orcid-with-rorcid/ for inspiring some of the structure and ideas throughout this document. I highly recommend reading it.

# Licensing

This walkthrough is distributed under a [https://creativecommons.org/licenses/by/4.0/](Creative Commons Attribution 4.0 International (CC BY 4.0) License).

<div>
    <br>
    <a>
        <img src="images/ccby.png" style="width: 400px;">
    </a>
</div>

# Load packages
When you download R it already has a number of functions built in: these encompass what is called **Base R**. However, many R users write their own libraries of functions, package them together in R **packages**, and provide them to the R community at no charge. This extends the capacity of R and allows us to do much more. In many cases, they improve on the Base R functions by making them easier and more straight-forward to use. In addition to `rorcid`, we will also be using the `dplyr`, `purrr`, `tidyr`, `anytime`, `lubridate`, and `janitor` packages. Some of these packages are part of the [tidyverse](https://www.tidyverse.org/), a collection of R packages designed for data science.

If you are using R and R Studio, you will need to use `install.packages()` function to install the packages first. We have already installed the packages here in our Binder repository, so we will simply load them by calling `library()`. Let's also set an option to see a max number of 100 columns and max 20 rows in our Jupyter Notebooks environment, to make printed tables easier to look at.

In [2]:
#load packages
library(tidyr)
library(purrr)
library(lubridate)
library(rorcid)
library(anytime)
library(httr)
library(janitor)

# increase number of columns and rows displayed when we print a table
options(repr.matrix.max.cols=100, repr.matrix.max.rows=20)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




## Setting up rorcid

If you haven't done so already, create an ORCID account at https://orcid.org/signin. If you have an ORCID but can't remember it, search for your name at https://orcid.org. If you try to sign in with an email address already associated with an ORCID, you'll be prompted to sign into the existing record. If you try to register with a different address, when you enter your name you'll be asked to review existing records with that name and verify that none of them belong to you--[see more on duplicate ORCID records](https://support.orcid.org/hc/en-us/articles/360006972593-How-do-you-check-for-duplicate-ORCID-records-). Make sure you have verified your email address.

Next, you need to set up an ORCID API client in order to get a token you can use in your API queries. According to the [ORCID API tutorial](https://members.orcid.org/api/tutorial/read-orcid-records), anyone can receive a key to access the public API--you do not have to belong to a member institution.

If you were using R Studio, this would be simple: call `orcid_auth()` and a browser window will open, giving you your API token and allowing you to store it in your R session. See [this step on the rorcid page of the course website](https://ciakovx.github.io/NEED TO FILL THE REST OF THIS URL) for instructions on how to do that. Here in our Jupyter Notebook, we will be following the steps at <https://support.orcid.org/hc/en-us/articles/360006897174>:

1. Sign in to your ORCID account
2. In the upper right corner, click your name, then in the drop-down menu, click **Developer Tools**. Note: In order to access Developer Tools, you must verify your email address. If you have not already verified your email address, you will be prompted to do so at this point.
4. Click the **"Register for the free ORCID public API"** button
5. Review and agree to the terms of service when prompted.
6. Add your name in the Name field, https://www.orcid.org in the Your Website URL field, "Getting public API key" in Description field, and https://www.orcid.org in the redirect URI field. Click the diskette button to save.
7. A gray box will appear including your Client ID and Client Secret. In the below code chunk, copy and paste the client ID and the client secret respectively. Make sure to leave the quotation marks (e.g. `orcid_client_id <- "APP-FDFJKDSLF320SDFF"` and `orcid_client_secret <- "c8e987sa-0b9c-82ed-91as-1112b24234e"`). Then execute the code chunk.

In [2]:
# copy/paste your client ID from https://orcid.org/developer-tools
orcid_client_id <- "APP-UXL72QIF91OSUWAN"

# copy/paste your client secret from https://orcid.org/developer-tools
orcid_client_secret <- "c7e034dc-0b9c-48cf-98a3-24446b22792e"

8. Now execute the below code, which will send a `POST` request (from the `httr` package) to ORCID and return to you an access token.

In [4]:
orcid_request <- POST(url  = "https://orcid.org/oauth/token",
          config = add_headers(`Accept` = "application/json",
                               `Content-Type` = "application/x-www-form-urlencoded"),
          body = list(grant_type = "client_credentials",
                      scope = "/read-public",
                      client_id = orcid_client_id,
                      client_secret = orcid_client_secret),
          encode = "form")

Now that we have the response, we can use `content` from `httr` to get the information we want. We will then `print()` that to the console to see the contents.

In [7]:
orcid_response <- content(orcid_request)
print(orcid_response)

$access_token
[1] "e9400606-e4f8-4279-a91c-b7eff520becc"

$token_type
[1] "bearer"

$refresh_token
[1] "461701fc-0248-41c9-9b78-0f177722349f"

$expires_in
[1] 631138518

$scope
[1] "/read-public"

$orcid
NULL



We now use `Sys.setenv` to set our `ORCID_TOKEN` to the `access_token` value. This will give us the necessary authorization to make API calls to gather public ORCID records information.

In [5]:
Sys.setenv(ORCID_TOKEN = orcid_response$access_token)

Finally, we use `orcid_auth()` to confirm that we have set our token properly. If so, it will print `Bearer` and the access token we just assigned. We are ready to use `rorcid`!

In [6]:
rorcid::orcid_auth(scope = "/authenticate",
                   reauth = TRUE)

[1] "Bearer e9400606-e4f8-4279-a91c-b7eff520becc"

After you have your access token, you only ever need to complete the `Sys.setenv()` step and the `rorcid::orcid_auth()` step. Your access token is good for several years. Just save it somewhere and reuse as needed.

# Finding ORCID iDs with `rorcid::orcid_search()`

The `rorcid::orcid_search()` function takes a query and returns a data frame with three columns: first name, last name, and ORCID iD. We use this when we have some data about a person or people, and we want to get their ORCID iDs. 

Call `?orcid_search` to view the available parameters.

For this example, we will use the fictitious professor [Josiah S(tinkney) Carberry](https://library.brown.edu/hay/carberry.php), "legendary professor of psychoceramics (the study of cracked pots) since 1929," at Brown University. Despite being make-believe, Carberry has a profile in ORCID that can be used for test cases such as these.

## Names

We start with a simple search by family name with the `family_name` argument.

In [12]:
carberry <- rorcid::orcid_search(family_name = 'carberry')
carberry

first,last,orcid
<chr>,<chr>,<chr>
Caroline,Carberry,0000-0001-8276-3703
Amanda,Carberry,0000-0001-5591-5599
Adrian,Carberry,0000-0003-0362-0689
Emma,Carberry,0000-0002-9770-3460
Josie,Carberry,0000-0002-4463-641X
peter,carberry,0000-0002-3555-2193
David,Carberry,0000-0002-1436-4089
Susan,Carberry,0000-0001-7710-0796
Paul,Carberry,0000-0002-3409-0724


Looking at the data frame in your R environment, you will see it returns 10 observations of three variables. By default, `orcid_search` returns a limit of 10. We can increase the number of results returned by using the `rows` argument:

In [13]:
carberry <- rorcid::orcid_search(family_name = 'carberry',
                                 rows = 50)
carberry

first,last,orcid
<chr>,<chr>,<chr>
Caroline,Carberry,0000-0001-8276-3703
Amanda,Carberry,0000-0001-5591-5599
Adrian,Carberry,0000-0003-0362-0689
Emma,Carberry,0000-0002-9770-3460
Josie,Carberry,0000-0002-4463-641X
peter,carberry,0000-0002-3555-2193
David,Carberry,0000-0002-1436-4089
Susan,Carberry,0000-0001-7710-0796
Paul,Carberry,0000-0002-3409-0724


We now get 28 results. However, we need not look through that long list to find Josiah, but can add another argument: `given_name`. Multiple arguments are combined with AND, such that the above example gets passed to ORCID as **given-names:josiah** AND **family-name:carberry**.

In [14]:
carberry <- rorcid::orcid_search(given_name = 'josiah',
                                 family_name = 'carberry')
carberry

first,last,orcid
<chr>,<chr>,<chr>
Josiah,Carberry,0000-0002-1028-6941
Josiah,Carberry,0000-0002-1825-0097


It looks like there are two people in the public ORCID registry with the first name Josiah and the last name Carberry. 
In R Studio, we can launch the actual ORCID profiles to our browser by using `rorcid::browse()` function, where we use brackets to look up the first and second items in the `orcid` variable--i.e. `rorcid::browse(carberry$orcid[1])` however, we can't do that in Jupyter Notebooks, so we will simply look them up manually: <http://orcid.org/0000-0002-1028-6941> and <http://orcid.org/0000-0002-1825-0097>.

Call `rorcid:::field_match_list` to see a map of arguments.

In [18]:
rorcid::fields

field,description,path
<fct>,<fct>,<fct>
orcid,The ORCID identifier for the researcher or contributor,//orcid-profile/orcid
given-names,The given names of the researcher of contributor,//orcid-profile/orcid-bio/personal-details/given-names
family-name,The family name of the researcher of contributor,//orcid-profile/orcid-bio/personal-details/family-name
past-institution-affiliation-name,The name of any past institution in the researcher or contributors profile,"//orcid-profile/orcid-bio/affiliations/affiliation[affiliation-type=""past-institution""]/affiliation-name"
current-primary-institution-affiliation-name,The name of the primary institution of the researcher or contributor,"//orcid-profile/orcid-bio/affiliations/affiliation[affiliation-type=""current-primary-institution""]/affiliation-name"
current-institution-affiliation-name,The name of non-primary institutions of the researcher or contributor,"//orcid-profile/orcid-bio/affiliations/affiliation[affiliation-type=""current-institution""]/affiliation-name"
credit-name,The name that normally appears on publications by the researcher or contributor,//orcid-profile/orcid-bio/personal-details/credit-name
other-names,Alternative names that may have appeared on publications by the researcher or contributor,//orcid-profile/orcid-bio/personal-details/other-names
email,The email address of the researcher or contributor,//orcid-profile/orcid-bio/contact-details/email


---
**TRY IT YOURSELF**

1. Do a name search for yourself or a colleague. Try it with and without `given_names` and `family_names`. Increase or decrease the `rows` returned.

In [ ]:
# run an orcid name search




## Affiliation name

`orcid_search` includes the argument `affiliation_org`, which searches across all of one's affiliation data (employment, education, invited positions, membership & service). Because this is such a broad search, it has the potential to return false positives if you are using it on it's own.

In [20]:
carberry <- rorcid::orcid_search(family_name = 'carberry', 
                                 affiliation_org = 'Wesleyan')
carberry

first,last,orcid
<chr>,<chr>,<chr>
Josiah,Carberry,0000-0002-1825-0097


There are also arguments for past institution (`past_inst`) and current institution (`current_inst`), as well as institutional identifiers (see below).

---
**TRY IT YOURSELF**

1. If your ORCID profile has affiliation data, run a name and affiliation search. If you have designated current and past institutions, try both:

In [21]:
# run a name and affiliation search



## Email address

We can search by email address, however, [according to ORCID](https://members.orcid.org/api/tutorial/search-orcid-registry), as of February 2017, fewer than 2% of the 3+ million email addresses on ORCID records are public, so this one may not be incredibly helpful.  Since Josiah doesn't have email, we'll use mine.

In [22]:
clarke <- rorcid::orcid_search(email = 'clarke.iakovakis@okstate.edu')
clarke

first,last,orcid
<chr>,<chr>,<chr>
Clarke,Iakovakis,0000-0002-9260-8456


If the person has chosen to keep their email address private, the function will return an empty dataframe.`


## Keywords

If the individual has added keywords to their ORCID profile, we can search those. Dr. Carberry's profile includes the keyword "psychoceramics" (the study of cracked pots) (*note: this function is currently under development - :

In [23]:
carberry <- rorcid::orcid_search(family_name = 'carberry',
                                 keywords = 'psychoceramics')
carberry

first,last,orcid
<chr>,<chr>,<chr>
Josiah,Carberry,0000-0002-1825-0097


## Work title and DOI

If you know the name of a work (i.e. article, book chapter, etc.) or its DOI, you can obtain the associated ORCID iD by using the `work_title` or `digital_object_ids` **if and only if** the authos have added it to their ORCID profile.

We will search for the article ["Building Software Building Community: Lessons from the rOpenSci Project"](https://openresearchsoftware.metajnl.com/articles/10.5334/jors.bu/print/). Notice how the title is in double quotes, inside of single quotes, and the colon is removed.

In [26]:
ropensci1 <- rorcid::orcid_search(work_title = '"Building Software Building Community"')
ropensci1

first,last,orcid
<chr>,<chr>,<chr>
Edmund,Hart,0000-0001-7367-7969
Scott,Chamberlain,0000-0003-1444-9135


This gives us two authors: Edmund Hart and Scott Chamberlain. Notice we get a different result when we look the same article up by DOI:

In [27]:
ropensci2 <- rorcid::orcid_search(digital_object_ids = '"10.5334/jors.bu"')
ropensci2

first,last,orcid
<chr>,<chr>,<chr>
Edmund,Hart,0000-0001-7367-7969


If we navigate to the author profiles on the web, we can see this is because Edmund Hart added the article to his ORCID profile, whereas Scott Chamberlain added the dataset (which has a different DOI).

Thus a word of caution when searching by article title or DOI: a significant amount of data in ORCID is manually added or added with incorrect, inconsistent, or incomplete metadata. In other words, the fact that you didn't get results or got erroneous results may not be due to errors in your queries, but rather errors in the data itself.

## Institutional ID (Ringgold, ISNI, GRID) - `ringgold-org-id:`

When filling out an ORCID profile, users are encouraged to select their institutions from the drop-down menu, which will ensure it includes the Ringgold ID and any other unique identifiers, such as ISNI and GRID, that ORCID has for that institution. Read the ORCID report, ["Organization identifiers: current provider survey"](https://orcid.org/sites/default/files/ckfinder/userfiles/files/20161031%20OrgIDProviderSurvey.pdf) to learn more.

In [28]:
carberry <- rorcid::orcid_search(family_name = 'carberry',
                               ringgold_org_id = '5468')
carberry

first,last,orcid
<chr>,<chr>,<chr>
Josiah,Carberry,0000-0002-1825-0097


You have to [register with Ringgold](https://www.ringgold.com/identify-online-guests/) to search in their registry.  [GRID is open for searching](https://www.grid.ac/institutes/grid.268117.b). Unfortunately, most organizations in ORCID have a Ringgold and not a GRID.

Sometimes different entities on campus will have separate Ringgold IDs; you may consider contacting Ringgold to get the full list of your institution's identifiers.

## Name and email address domain

We can search by name and email address domain by using an asterisk followed by the domain name

In [29]:
clarke <- rorcid::orcid_search(family_name = 'iakovakis',
                               email = '*@okstate.edu')
clarke

first,last,orcid
<chr>,<chr>,<chr>
Clarke,Iakovakis,0000-0002-9260-8456


## Advanced searching

`orcid_search` is a wrapper for another `rorcid` function--`orcid()`, which allows for a more advanced range of searching, including Boolean OR operators.

According to `help(orcid)`:

> You can use any of the following within the query statement: given-names, family-name, credit-name, other-names, email, grant-number, patent-number, keyword, worktitle, digital-objectids, current-institution, affiliation-name, current-primary-institution, text, past-institution, peer-review-type, peer-review-role, peer-review-group-id, biography, external-id-type-and-value

Note that `current_prim_inst` and `patent_number` parameters have been removed as ORCID has removed them.

### Searching with Boolean OR

We can combine affiliation names, Ringgold IDs, and email addresses using the `OR` operator to cover all our bases, in case the person or people we are looking for did not hit on of those values. This will return all records that either have a Ringgold of 7618, have an affiliation name of "Oklahoma State," or have an email domain ending in "\@okstate.edu."

In [30]:
clarke <- rorcid::orcid(query = 'family-name:iakovakis AND(ringgold-org-id:7618 OR
                           email:*@okstate.edu OR 
                           affiliation-org-name:"Oklahoma State")')
clarke

,orcid-identifier.uri,orcid-identifier.path,orcid-identifier.host
,<chr>,<chr>,<chr>
1,https://orcid.org/0000-0002-9260-8456,0000-0002-9260-8456,orcid.org


This can also be helpful if you want to cast a very wide net and capture everyone affiliated with your institution who has an ORICID iD. Keep in mind that this searches across **all** of an individuals listed affiliations (employment, education, invited positions, membership & service) past and present. So it has **will** return false positives--in other words, one should not use it to get ORCID iDs of all individuals currently at an institution, because it will include those who previously worked there or got their degree from there.

The maximum number of returned results is 200, which we can modify with the `rows` argument:

In [ ]:
my_osu_orcids <- rorcid::orcid(query = 'ringgold-org-id:7618 OR email:*@okstate.edu OR
                               affiliation-org-name:"Oklahoma State"',
                               rows = 25)
my_osu_orcids

If you want to retrieve a complete set of all results above 200, we have to write a small function. First, we will wrap our API call in `base::attr` and include a `"found"` argument to see how many results are found with that call:

In [ ]:
my_osu_orcid_count <- base::attr(rorcid::orcid(query = 'ringgold-org-id:7618 OR 
                               email:*@okstate.edu OR affiliation-org-name:"Oklahoma State"'),
                      "found")
my_osu_orcid_count

There are 1,897 records at the time of writing. Next we will first create a numeric vector using `seq` that starts with 0 and ends with 1897, adding 200 to each value incrementally.

In [ ]:
my_pages <- seq(from = 0, to = my_osu_orcid_count, by = 200)
my_pages

Finally, we will write a small function using `map` from the `purrr` package. In essence, this takes each value from our `my_pages` vector, and passes it into the `page` argument of the `orcid()` query. In other words, the first loop will get results 0-200, the next will get 200-400, and so on.

In [ ]:
my_osu_orcids <- purrr::map(
  my_pages,
  function(page) {
  print(page)
  my_orcids <- rorcid::orcid(query = 'ringgold-org-id:7618 OR 
                               email:*@okstate.edu OR affiliation-org-name:"Oklahoma State"',
                               rows = 200,
                               start = page)
  return(my_orcids)
  })

We can then use the `bind_rows()` function from `dplyr` to pull the data together into a single dataframe, and the `clean_names()` function from `janitor`, described below, to make the column names easier to handle. We also introduce here the Pipe Operator, which is also described below.

In [ ]:
my_osu_orcids_data <- my_osu_orcids %>%
  dplyr::bind_rows() %>%
  janitor::clean_names()

Of course, these are only the ORCID iDs, with no other data. The next sections will describe other functions in `rorcid` to get biographical, employment, and works data from the profiles.

# `clean_names()` and `%>%`

Often times the columns returned from the ORCID API have a complicated combination of punctuation that can make them hard to use. The `clean_names()` function from the `janitor` package is optional and used only to simplify the column names of the data. It converts all punctuation to underscores, so the field `orcid-identifier.uri` becomes `orcid_identifier_uri`.

A [Pipe Operator](https://www.datacamp.com/community/tutorials/pipe-r-tutorial) `%>%`. A pipe takes the output of one statement and makes it the input of the next statement. You can think of it as "then" in natural language. So the above script first runs the `orcid()` API call, then it clean the column names of the data that was pulled into R as a result of that call. So for example, in the expression above, we first call up the `my_osu_orcids` data. We then apply the `bind_rows()` function to pull all the data together into a single data frame, and then clean the names with `clean_names()`.

# Finding biographical information

The `orcid()` function gets the IDs, but no information about the person. For that, you will need to use `orcid_person()`.

Unlike `orcid()`, `orcid_person()` does not take a query; it accepts only ORICID iDs in the form XXXX-XXXX-XXXX-XXXX. So we can get the ORICID iD itself into it's own vector. We can then pass that argument on to `orcid_person()`.

In [ ]:
carberry_orcid <- "0000-0002-1825-0097"
carberry_person <- rorcid::orcid_person(carberry_orcid)

If you look at the result for carberry_person in the Environment Pane in R Studio, you will see it returned a List of 1. We can view it here on this website using the `listviewer` package.

In [ ]:
listviewer::jsonedit(carberry_person, mode = "view")

Click the drop-down arrow next to the ORICID iD. We see one list here (his ORCID iD) and inside of that list is more lists. And inside those lists is even more lists! We can see the names of the top-level elements of the list by running `names(carberry_person[[1]])`.

In [ ]:
names(carberry_person[[1]])

If you click the drop-down arrow next to name. You can run `names(carberry_person[[1]]$name)` to see the names of those elements. While this is great data, if we want to run some analysis on it, we need to get it into a nice, tidy data frame. 

## Getting the data into a data frame

This is not an easy or straightforward process. I provide below one strategy to get some of the relevant data, using `map` functions from the `purrr` package and building a `tibble` (the `tidyverse`'s more efficient data frame) piece by piece.

In [ ]:
carberry_data <- carberry_person %>% {
    dplyr::tibble(
      created_date = purrr::map_dbl(., purrr::pluck, "name", "created-date", "value", .default=NA_character_),
      given_name = purrr::map_chr(., purrr::pluck, "name", "given-names", "value", .default=NA_character_),
      family_name = purrr::map_chr(., purrr::pluck, "name", "family-name", "value", .default=NA_character_),
      credit_name = purrr::map_chr(., purrr::pluck, "name", "credit-name", "value", .default=NA_character_),
      other_names = purrr::map(., purrr::pluck, "other-names", "other-name", "content", .default=NA_character_),
      orcid_identifier_path = purrr::map_chr(., purrr::pluck, "name", "path", .default = NA_character_),
      biography = purrr::map_chr(., purrr::pluck, "biography", "content", .default=NA_character_),
      researcher_urls = purrr::map(., purrr::pluck, "researcher-urls", "researcher-url", .default=NA_character_),
      emails = purrr::map(., purrr::pluck, "emails", "email", "email", .default=NA_character_),
      keywords = purrr::map(., purrr::pluck, "keywords", "keyword", "content", .default=NA_character_),
      external_ids = purrr::map(., purrr::pluck, "external-identifiers", "external-identifier", .default=NA_character_)
    )
  }
carberry_data

* The **created_date** comes from within the name list, so technically it is the date the name was created, not the date the ORCID account was created, which is not available in this data. This is plucked with `map_dbl()` because it is in `double` format (a numeric data type in R) . 
* The **given_name**, **family_name**, **credit_name**, **orcid_identifier_path**, and **biography** are plucked with `map_chr()` because they are both `character` types.
* The **other_names**, **keywords**, **researcher_urls**, and **external_ids** are plucked with `map()` because there may be multiple values (unlike the other names, in which you can only have one). For example, someone may have multiple other names, or multiple keywords. So this will return a nested list to the tibble; we will discuss below how to unnest it.

Each of these functions includes a `.default = NA_character_` argument because if the value is NULL (if the ORCID author didn't input the information) then it will convert that NULL to NA.

## Fixing dates

View the created date by running `carberry_data$created_date` and you will see this is a number, not a date:

In [ ]:
carberry_data$created_date

The dates are in [Unix time](https://en.wikipedia.org/wiki/Unix_time), which is the number of seconds that have elapsed since January 1, 1970. In ORCID, this is in milliseconds. We can use the `anytime()` function from the `anytime` package created by Dirk Eddelbuettel to convert it and return a POSIXct object. You have to divide it by 1000 because it's in milliseconds. Below we use the `mutate()` function from `dplyr` to overwrite the `created_date` and `last_modified_date` UNIX time with the human readable POSIXct dates.

In [ ]:
carberry_datesAndTimes <- carberry_data %>%
  dplyr::mutate(created_date = anytime::anytime(created_date/1000))
carberry_datesAndTimes$created_date

That looks much better: April 15th, 2016 at 5:00 PM and 17 seconds Central Daylight Time.

If you'd prefer to do away with the time altogether (and keep only the month/day/year), you can use `anydate()` instead of `anytime()`.

In [ ]:
carberry_datesOnly <- carberry_data %>%
  dplyr::mutate(created_date = anytime::anydate(created_date/1000))
carberry_datesOnly$created_date

Check out the `lubridate` package for more you can do with dates. It is installed with `tidyverse`, but not loaded, so you have to load it with its own call to `library()` (we did this at the beginning of the session). For example, you may be more interested in year of creation than month. So after you run the conversion with `anytime`, you can create year variables with `mutate()`:

In [ ]:
carberry_years <- carberry_datesOnly %>%
  dplyr::mutate(created_year = lubridate::year(created_date))
carberry_years$created_year

## Unnesting nested lists

There are nested lists in this data frame that can be unnested. The **other_names** and **keywords** values are character vectors, while the **researcher_urls** and **external_ids** values are data frames themselves. We can use the `unnest()` function from the `tidyr` package to unnest both types. In other words, this will make each element of the list its own row. For instance, since there are two keywords for carberry ("psychoceramics" and "ionian philology"), there will now be two rows that are otherwise identical except for the keywords column:

In [ ]:
carberry_keywords <- carberry_data %>%
  tidyr::unnest(keywords)
carberry_keywords$keywords

We can see which columns are lists by calling `is_list()` in the `map_lgl()` function (this will return a TRUE/FALSE for each column that is a list), and subsetting the `names()` of `carberry_data` by those values:

In [ ]:
carberry_list_columns <- map_lgl(carberry_data, is_list)
names(carberry_data)[carberry_list_columns]

Rather than having 1 observation of 11 variables, the data frame now has 2 observations of 7 variables. We know why there are two observations (because there are 2 keywords), but why are there fewer variables? Because there is an argument to `unnest()` called `.drop`, which is set to `TRUE` by default, meaning all additional list columns will be dropped. If you want to keep them, just set it to `FALSE` Note, however, that it will not unnest them.

In [ ]:
carberry_keywords <- carberry_data %>%
  tidyr::unnest(keywords, .drop = FALSE)
carberry_keywords

You can unnest multiple nested columns, but keep in mind that this will multiply the duplicated columns in your data frame, because there will be it is spreading the key-value pairs across multiple columns. For more on wide and long data, read Hadley Wickham's paper ["Tidy data,"](https://vita.had.co.nz/papers/tidy-data.html) published in *The Journal of Statistical Software.*

In [ ]:
carberry_keywords_otherNames <- carberry_data %>%
  tidyr::unnest(keywords, .drop = FALSE) %>%
  tidyr::unnest(other_names, .drop = FALSE)
carberry_keywords_otherNames

When we unnest *researcher_urls* or *external_ids*, we will see many more columns added. That is because each of these nested lists contains multiple variables:

In [ ]:
carberry_researcherURLs <- carberry_data %>%
  tidyr::unnest(researcher_urls, .drop = FALSE)
carberry_researcherURLs

Carberry has two URLs: his Wikipedia page and a page about him on the Brown University Library. So a row is created for each of these URLs, and multiple columns are added such as the last modified date, the url value, and so on. You can keep or remove columns you don't want using `select()` from the `dplyr` package.

# Writing to CSV

We will use the `write_csv()` function from the `readr` package to write our data to disk. This package was loaded when you called `library(tidyverse)` at the beginning of the session.

With a typical data frame, you can simply write the `carberry_data` data frame to a CSV with the following code:

In [ ]:
write_csv(carberry_keywords, "C:/Users/MyUserName/Desktop/carberry_data.csv")

The problem is, due to the nested lists we described above, R will throw an error: `"Error in stream_delim_(df, path, ...) : Don't know how to handle vector of type list."`

You have a few choices:

1. You can unnest one of the columns and leave `.drop` set to `TRUE`. This will add rows for all the values in the nested lists, and drop the additional nested lists. 

Replace "MyUserName" below with your actual user name to write this file to your desktop. You can get your username by calling `Sys.getenv("USERNAME")`.

In [ ]:
my_user_name <- Sys.getenv("USERNAME")
carberry_keywords <- carberry_data %>%
  tidyr::unnest(keywords)
write_csv(carberry_keywords, file.path("C:/Users", my_user_name, "Desktop/carberry_data1.csv"))

2. You can drop the nested lists altogether using a combination of `select_if()` from `dplyr` and `negate()` from `purrr` to drop all lists in the data frame. This is essentially saying, only keep the columns that are not lists. In this example, the number of variables falls to 6, since we have 5 list columns.

In [ ]:
carberry_data_short <- carberry_data %>%
  dplyr::select_if(purrr::negate(is_list))

3. You can use `mutate()` from `dplyr` to coerce the list columns into character vectors.

In [ ]:
carberry_data_mutated <- carberry_data %>%
  dplyr::mutate(keywords = as.character(keywords)) %>%
  dplyr::mutate(other_names = as.character(other_names)) %>%
  dplyr::mutate(researcher_urls = as.character(map(carberry_data$researcher_urls, purrr::pluck, "url.value", .default=NA_character_))) %>%
  dplyr::mutate(external_ids = as.character(map(carberry_data$external_ids, purrr::pluck, "external-id-url.value", .default=NA_character_)))
write_csv(carberry_data_mutated, "C:/Users/MyUserName/Desktop/carberry_data2.csv")

# Getting data on multiple people with `orcid_person()`

## Searching by ORICID iDs

`orcid_person()` is vectorized, so you can pass in multiple ORICID iDs and it will return a list of results for each ID, with each element named by the ORICID iD.

In [ ]:
my_orcids <- c("0000-0002-1825-0097", "0000-0002-9260-8456")
my_orcid_person <- rorcid::orcid_person(my_orcids)
listviewer::jsonedit(my_orcid_person, mode = "view")

We see that we are given a list of 2, each containing the person data. We can put this into a data frame using the same code as above.

In [ ]:
my_orcid_person_data <- my_orcid_person %>% {
    dplyr::tibble(
      created_date = purrr::map_dbl(., purrr::pluck, "name", "created-date", "value", .default=NA_character_),
      given_name = purrr::map_chr(., purrr::pluck, "name", "given-names", "value", .default=NA_character_),
      family_name = purrr::map_chr(., purrr::pluck, "name", "family-name", "value", .default=NA_character_),
      credit_name = purrr::map_chr(., purrr::pluck, "name", "credit-name", "value", .default=NA_character_),
      other_names = purrr::map(., purrr::pluck, "other-names", "other-name", "content", .default=NA_character_),
      orcid_identifier_path = purrr::map_chr(., purrr::pluck, "name", "path", .default = NA_character_),
      biography = purrr::map_chr(., purrr::pluck, "biography", "content", .default=NA_character_),
      researcher_urls = purrr::map(., purrr::pluck, "researcher-urls", "researcher-url", .default=NA_character_),
      emails = purrr::map(., purrr::pluck, "emails", "email", "email", .default=NA_character_),
      keywords = purrr::map(., purrr::pluck, "keywords", "keyword", "content", .default=NA_character_),
      external_ids = purrr::map(., purrr::pluck, "external-identifiers", "external-identifier", .default=NA_character_)
    )
  } %>%
  dplyr::mutate(created_date = anytime::anydate(created_date/1000))
my_orcid_person_data

We now have a nice, neat dataframe of both people's ORCID name data.

## Searching by names

When we want data on multiple people and have only their names, we can build a query. 

Now we can build a query that will work with the `given-names:` and `family-name:` arguments to `query` in `orcid` in order to get the ORICID iDs:



In [3]:
profs <- tibble("FirstName" = c("Josiah", "Clarke"),
                     "LastName" = c("Carberry", "Iakovakis"))
profs
orcid_query <- paste0("given-names:",
                      profs$FirstName,
                      " AND family-name:",
                      profs$LastName)
orcid_query

FirstName,LastName
<chr>,<chr>
Josiah,Carberry
Clarke,Iakovakis


[1] "given-names:Josiah AND family-name:Carberry" 
[2] "given-names:Clarke AND family-name:Iakovakis"

This returns a vector with two queries formatted for nice insertion into `rorcid::orcid()`. We can use `purr::map()` to create a loop. What this is saying is, take each element of `orcid_query` and run a function with it that prints it to the console and runs `rorcid::orcid()` on it, then return each result to `my_orcids_list().` This returns a list of two items. We can then wrap `as_tibble()` in `map_dfr` to create a data frame from those list elements.


In [7]:
my_orcids_df <- purrr::map(
  orcid_query,
  function(x) {
    print(x)
    orc <- rorcid::orcid(x)
  }
  ) %>%
    map_dfr(., as_tibble) %>%
  janitor::clean_names()
my_orcids_df

[1] "given-names:Josiah AND family-name:Carberry"
[1] "given-names:Clarke AND family-name:Iakovakis"


orcid_identifier_uri,orcid_identifier_path,orcid_identifier_host
<chr>,<chr>,<chr>
https://orcid.org/0000-0002-1028-6941,0000-0002-1028-6941,orcid.org
https://orcid.org/0000-0002-1825-0097,0000-0002-1825-0097,orcid.org
https://orcid.org/0000-0002-9260-8456,0000-0002-9260-8456,orcid.org


First we want to remove the Carberry row that we don't want (remember that there are two Carberry ORCID accounts, and one doesn't have much data in it). We can do this using the `filter()` function from `dplyr` and the `!=` symbol, which is equivalent to "is not equal to."


In [9]:
my_orcids_df <- my_orcids_df %>%
  dplyr::filter(orcid_identifier_path != "0000-0002-1028-6941")
my_orcids_df

orcid_identifier_uri,orcid_identifier_path,orcid_identifier_host
<chr>,<chr>,<chr>
https://orcid.org/0000-0002-1825-0097,0000-0002-1825-0097,orcid.org
https://orcid.org/0000-0002-9260-8456,0000-0002-9260-8456,orcid.org


This is a data frame of two items. , grab the ORICID iDs, and run the same function we ran above in order to get the name data and the IDs into a single data frame. 


In [12]:
my_orcids <- my_orcids_df$orcid_identifier_path
my_orcid_person <- rorcid::orcid_person(my_orcids)
my_orcid_person_data <- my_orcid_person %>% {
    dplyr::tibble(
      created_date = purrr::map_dbl(., purrr::pluck, "name", "created-date", "value", .default=NA_character_),
      given_name = purrr::map_chr(., purrr::pluck, "name", "given-names", "value", .default=NA_character_),
      family_name = purrr::map_chr(., purrr::pluck, "name", "family-name", "value", .default=NA_character_),
      credit_name = purrr::map_chr(., purrr::pluck, "name", "credit-name", "value", .default=NA_character_),
      other_names = purrr::map(., purrr::pluck, "other-names", "other-name", "content", .default=NA_character_),
      orcid_identifier_path = purrr::map_chr(., purrr::pluck, "name", "path", .default = NA_character_),
      biography = purrr::map_chr(., purrr::pluck, "biography", "content", .default=NA_character_),
      researcher_urls = purrr::map(., purrr::pluck, "researcher-urls", "researcher-url", .default=NA_character_),
      emails = purrr::map(., purrr::pluck, "emails", "email", "email", .default=NA_character_),
      keywords = purrr::map(., purrr::pluck, "keywords", "keyword", "content", .default=NA_character_),
      external_ids = purrr::map(., purrr::pluck, "external-identifiers", "external-identifier", .default=NA_character_)
    )
} %>%
  dplyr::mutate(created_date = anytime::anydate(created_date/1000))
my_orcid_person_data

created_date,given_name,family_name,credit_name,other_names,orcid_identifier_path,biography,researcher_urls,emails,keywords,external_ids
<date>,<chr>,<chr>,<chr>,<named list>,<chr>,<chr>,<named list>,<named list>,<named list>,<named list>
2016-04-15,Josiah,Carberry,NA,"Josiah Stinkney Carberry, J. Carberry , J. S. Carberry",0000-0002-1825-0097,"Josiah Carberry is a fictitious person. This account is used as a demonstration account by ORCID, CrossRef and others who wish to demonstrate the interaction of ORCID with other scholarly communication systems without having to use a real-person's account. Josiah Stinkney Carberry is a fictional professor, created as a joke in 1929. He is said to still teach at Brown University, and to be known for his work in ""psychoceramics"", the supposed study of ""cracked pots"". See his Wikipedia entry for more details.","Brown University Page , Wikipedia Entry , public , public , /0000-0002-1825-0097/researcher-urls/568395 , /0000-0002-1825-0097/researcher-urls/568394 , 568395 , 568394 , 2 , 1 , 1446663146890 , 1446663146889 , 1462157645967 , 1462157645967 , NA , NA , NA , NA , NA , NA , NA , NA , https://orcid.org/0000-0002-1825-0097 , https://orcid.org/0000-0002-1825-0097 , 0000-0002-1825-0097 , 0000-0002-1825-0097 , orcid.org , orcid.org , Josiah Carberry , Josiah Carberry , http://library.brown.edu/about/hay/carberry.php, http://en.wikipedia.org/wiki/Josiah_Carberry",NA,"psychoceramics , ionian philology","Scopus Author ID , 7007156898 , self , public , /0000-0002-1825-0097/external-identifiers/698979 , 698979 , 0 , 1494016313820 , 1494016313820 , NA , NA , https://orcid.org/client/0000-0002-5982-8983 , 0000-0002-5982-8983 , orcid.org , Scopus - Elsevier , https://orcid.org/0000-0002-1825-0097 , 0000-0002-1825-0097 , orcid.org , Josiah Carberry , http://www.scopus.com/inward/authorDetails.url?authorID=7007156898&partnerID=MN8TOARS"
2016-04-15,Clarke,Iakovakis,NA,"Clarke L. Iakovakis, C. Iakovakis , C.L. Iakovakis",0000-0002-9260-8456,"I am the Scholarly Services Librarian at Oklahoma State University. I received my MS in Information Studies from the University of Texas at Austin, and my BA in History from Texas State University.","figshare page , Oklahoma State University profile , Google Scholar Profile , public , public , public , /0000-0002-9260-8456/researcher-urls/1719866 , /0000-0002-9260-8456/researcher-urls/1719867 , /0000-0002-9260-8456/researcher-urls/1719868 , 1719866 , 1719867 , 1719868 , 3 , 2 , 1 , 1556293653296 , 1556293680922 , 1556293746863 , 1556293746865 , 1556293746865 , 1556293746863 , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , https://orcid.org/0000-0002-9260-8456 , https://orcid.org/0000-0002-9260-8456 , https://orcid.org/0000-0002-9260-8456 , 0000-0002-9260-8456 , 0000-0002-9260-8456 , 0000-0002-9260-8456 , orcid.org , orcid.org , orcid.org , Clarke Iakovakis , Clarke Iakovakis , Clarke Iakovakis , https://figshare.com/authors/Clarke_Iakovakis/3990140 , https://info.library.okstate.edu/clarke-iakovakis , https://scholar.google.com/citations?user=V9m2_IAAAAAJ&hl=en&oi=sra",clarke.iakovakis@okstate.edu,"open access , scholarly communications, librarianship",NA


This will be exactly the same thing as we saw above, however we got it from a simple vector of names.


# Splitting Names

If the names you have are not already separated into first and last name variables, here is a trick to do that:

Create a tibble using the `tibble()` function from the `dplyr` package. Then, use the `extract()` function from `tidyr`, along with some [regular expressions](https://www.rexegg.com/regex-quickstart.html), to create a first and last name variable:

In [13]:
my_names <- dplyr::tibble("name" = c("Josiah Carberry", "Clarke Iakovakis"))
my_names
my_clean_names <- my_names %>%
  tidyr::extract(name, c("FirstName", "LastName"), "([^ ]+) (.*)")
my_clean_names


name
<chr>
Josiah Carberry
Clarke Iakovakis


FirstName,LastName
<chr>,<chr>
Josiah,Carberry
Clarke,Iakovakis


## Unnesting

Again, we can unnest if we wish, knowing we'll multiply the number of rows even more now, because we have more values. For instance, if we unnest keywords, we'll now have 5 columns (2 keywords for carberry, and 3 keywords for iakovakis):

In [ ]:
my_orcid_person_keywords <- my_orcid_person_data %>%
  tidyr::unnest(keywords)
my_orcid_person_keywords

We can write this data to CSV using one of the three strategies outlined above. I'll use #3 and coerce all list columns to character.

In [ ]:
my_orcid_person_data_mutated <- my_orcid_person_data %>%
  dplyr::mutate(keywords = as.character(keywords)) %>%
  dplyr::mutate(other_names = as.character(other_names)) %>%
  dplyr::mutate(researcher_urls = map(my_orcid_person_data$researcher_urls, purrr::pluck, "url.value", .default=NA_character_)) %>%
  dplyr::mutate(external_ids = as.character(map(my_orcid_person_data$external_ids, purrr::pluck, "external-id-url.value", .default=NA_character_)))
write_csv(carberry_data_mutated, "C:/Users/MyUserName/Desktop/carberry_data3.csv")

# Getting employment data

In addition to biographical data, we can also get employment data with `orcid_employments()`.

## Getting employment data for an individual

In [ ]:
clarke_employment <- rorcid::orcid_employments(orcid = "0000-0002-9260-8456")
listviewer::jsonedit(clarke_employment, mode = "view")

Again it comes in a series of nested lists, but we'll just `pluck()` what we need and use `flatten_dfr()` to flatten the lists into a data frame. We will also use the `anydate()` function to go ahead and convert the dates while we're at it.

In [ ]:
clarke_employment_data <- clarke_employment %>%
  purrr::map(., purrr::pluck, "affiliation-group", "summaries") %>% 
  purrr::flatten_dfr() %>%
  janitor::clean_names() %>%
  dplyr::mutate(employment_summary_end_date = anytime::anydate(employment_summary_end_date/1000),
                employment_summary_created_date_value = anytime::anydate(employment_summary_created_date_value/1000),
                employment_summary_last_modified_date_value = anytime::anydate(employment_summary_last_modified_date_value/1000))

If you take a look at the data, you will see that there is no variable indicating whether these are current or past institutions of employment. In fact, the only way to check if the institution is a place of current employment is if the `employment_summary_end_date_year_value` is `NA`. Keep in mind that start and end dates are not required fields, and we can't be certain that people are updating their profiles. However, we can get a data frame of only those items meeting this criteria by using the `filter()` function from `dplyr`:

In [ ]:
clarke_employment_data_current <- clarke_employment_data %>%
  filter(is.na(employment_summary_end_date_year_value))

This will remove my previous two institutions, and keep only my current one: Oklahoma State University.

## Getting employment data for multiple people

Because `orcid_employments()` is vectorized, we can feed it multiple ORCID iDs and it will return data for the entire lot, if the individuals have added it. 

I'll grab a random assortment of OSU ORCID iDs from the previous section. Recall that these were pulled based on the detection of OSU data in either Ringgold, email, or affiliation names across all fields. We will issue the initial API call with `orcid_employments()`, then put it into a data frame with the same function we used above.

In [ ]:
my_osu_orcid_ids <- c("0000-0003-0599-2785", "0000-0002-8771-0368", "0000-0003-2863-6724", "0000-0001-6810-5560", "0000-0003-1935-9729", "0000-0002-9088-2312", "0000-0001-9792-7870", "0000-0003-3959-6916", "0000-0002-2621-5320", "0000-0001-9103-3040")
my_osu_employment <- rorcid::orcid_employments(my_osu_orcid_ids)
my_osu_employment_data <- my_osu_employment %>%
  purrr::map(., purrr::pluck, "affiliation-group", "summaries") %>% 
  purrr::flatten_dfr() %>%
  janitor::clean_names() %>%
  dplyr::mutate(employment_summary_end_date = anytime::anydate(employment_summary_end_date/1000),
                employment_summary_created_date_value = anytime::anydate(employment_summary_created_date_value/1000),
                employment_summary_last_modified_date_value = anytime::anydate(employment_summary_last_modified_date_value/1000))

First, let's rename some of the long names by removing some of the words. We'll do this using the `str_replace` function from `stringr`. You can think of this as analogous to Find + Replace in word processing. We take the `names()` of the data, and replace each of the phrases with nothing (i.e. the set of empty quotes).

In [ ]:
names(my_osu_employment_data) <- names(my_osu_employment_data) %>%
  stringr::str_replace(., "employment_summary_", "") %>%
  stringr::str_replace(., "source_source_", "") %>%
  stringr::str_replace(., "organization_disambiguated_", "")

Note that this may have multiple entries per person because it gathered their entire employment history. Now let's take a look at how many of these have Oklahoma State listed in their organization name:

In [ ]:
my_osu_employment_data_OSU <- my_osu_employment_data %>%
  filter(organization_name == "Oklahoma State University Stillwater")

Out of this set of 10 iDs, only 4 have OSU listed in their employment. The others may have achieved their degree from OSU, or done some service with OSU. Now, out of those 4, let's see how many are listed as current employees, as indicated with an `NA` in their `end_date_year_value`.

In [ ]:
my_osu_employment_data_OSU_current <- my_osu_employment_data_OSU %>%
  filter(is.na(end_date_year_value))

# Getting Works with `rorcid::works()` and `rorcid::orcid_works()`

## Getting works for an individual

There are two functions in `rorcid` to get all of the works associated with an ORICID iD: `orcid_works()` and `works()`. The main difference between these is `orcid_works()` returns a list, with each work as a list item, and each external identifier (e.g. ISSN, DOI) also as a list item. On the other hand, `works()` returns a nice, neat data frame that can be easily exported to a CSV. 

Like `orcid_person()`, these functions require an ORICID iD, and do not use the query fields we saw with the `orcid()` function.

In [ ]:
carberry_orcid <- c("0000-0002-1825-0097")
carberry_works <- rorcid::works(carberry_orcid) %>%
  as_tibble() %>%
  janitor::clean_names() %>%
  dplyr::mutate(created_date_value = anytime::anydate(created_date_value/1000))
carberry_works

Dr. Carberry has seven works. Because ORCID data can be manually entered, the integrity, completeness, and consistency of this data will sometimes vary. 

You can see the **external_ids_external_id** column is actually a nested list, a concept we discussed above. This can be unnested with the `tidyr::unnest()` function. Just as a single researcher can have multiple identifiers, a single work may also have multiple identifiers (e.g., DOI, ISSN, EID). If that is the case, when this column is unnested, there will be repeating rows for those items.

In [14]:
carberry_works_ids <- carberry_works %>%
  tidyr::unnest(external_ids_external_id) %>%
  janitor::clean_names()
carberry_works_ids

ERROR: Error in eval(lhs, parent, parent): object 'carberry_works' not found


In this case, we now have 13 observations of 27 variables rather than 7 observations of 24 variables. The extra rows are there because all but one of the works has two external identifiers. The extra columns are there because four new columns were added with the unnest (that's why we had to clean the names again): 

* **external_id_type** identifies the type of external identifier--see a list of [supported identifiers in ORCID](https://pub.orcid.org/v2.0/identifiers). (required)
* **external_id_value**: contains the identifier itself (required)
* **external_id_url:**  contains a link the identifier will resolve to (optional)
* **external_id_relationship**: indicates if the identifier refers to the item itself (`SELF`), such as a DOI or a person identifier, or a whole that the item is part of (`PART_OF`), such as an ISSN for a journal article.

So we can follow one of the three strategies outlined above if we want to write this to a CSV file: 1) unnest the column (as above), 2) drop the nested lists, or 3) mutate them into character vectors. 

## Getting works for multiple people

`orcid::works()` is not vectorized, meaning, if you have multiple ORICID iDs, you can't use it. Instead, you have to pass them to the `orcid::orcid_works()` function.

In [ ]:
my_orcids <- c("0000-0002-1825-0097", "0000-0002-9260-8456", "0000-0002-2771-9344")
my_works <- rorcid::orcid_works(my_orcids)
listviewer::jsonedit(my_works, mode = "view")

This returns a list of 3 elements, with the works nested in **group > work-summary**. They can be plucked and flattened into a data frame:

In [ ]:
my_works_data <- my_works %>%
  purrr::map_dfr(pluck, "works") %>%
  janitor::clean_names() %>%
  dplyr::mutate(created_date_value = anytime::anydate(created_date_value/1000))
my_works_data

## Unnesting external IDs

Now you may want to run some analysis using the external identifiers; for instance, you can use the `roadoi` package to look at which DOIs are open access.

We run into a problem here when we try to unnest the external IDs:

In [ ]:
my_works_externalIDs <- my_works_data %>%
  unnest(external_ids_external_id)

The error message reads: `"Error: Each column must either be a list of vectors or a list of data frames [external_ids_external_id]".` This is because some of the list columns are empty. We can just filter them out before unnesting:

In [ ]:
my_works_externalIDs <- my_works_data %>%
  dplyr::filter(!purrr::map_lgl(external_ids_external_id, purrr::is_empty)) %>%
  tidyr::unnest(external_ids_external_id)
my_works_externalIDs

If we want to keep them, there's a workaround: use `map_lgl` to first remove (`filter()` out) the `NULL` `external_id` columns, then `unnest` the ids, then bind back the `NULL` author columns, and finally deselecting the extra `author` and `link` columns as these are no longer in the transformed, unnested data.

In [ ]:
my_works_externalIDs_keep <- my_works_data %>% 
  dplyr::filter(!purrr::map_lgl(external_ids_external_id, purrr::is_empty)) %>% 
  tidyr::unnest(external_ids_external_id, .drop = TRUE) %>% 
  dplyr::bind_rows(my_works_data %>% 
                     dplyr::filter(map_lgl(external_ids_external_id, is.null)) %>%
                     dplyr::select(-external_ids_external_id))
my_works_externalIDs_keep

# Conclusion

The ORCID API is an excellent tool for analyzing research activity on multiple levels. `rorcid` makes gathering and cleaning the data easier. Thanks to both ORCID and Scott Chamberlain for their contributions to the community. Again, read Paul Oldham's excellent post at https://www.pauloldham.net/introduction-to-orcid-with-rorcid/ for more you can do. I hope this walkthrough  helps. If you need to get in touch with me, find my contact info at https://info.library.okstate.edu/clarke-iakovakis.